In [60]:
import numpy as np
import pandas as pd

pd.options.display.max_rows = 1000000 

%matplotlib inline
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["font.family"] = 'NanumBarunGothic'
plt.rcParams["font.size"] = 20
plt.rcParams["figure.figsize"] = (15,8)

mpl.rc('axes', labelsize=12)
mpl.rc('xtick', labelsize=8)
mpl.rc('ytick', labelsize=8)

import seaborn as sns


In [3]:
#읍면동 리스트
sample = pd.read_csv('data/2.세종시_표제부.csv')
sample['읍면동'] = sample['대지위치'].str.replace('세종특별자치시 ','')
sample['읍면동'] = sample['읍면동'].str.replace('조치원읍 ','조치원읍')
sample[['1', '2','3']] = sample['읍면동'].str.split(' ', n=2, expand=True)
sample['2'] = sample['2'].str.replace('조치원읍','조치원읍 ')
list_d = list(set(sample['2']))
data_d = pd.DataFrame(list_d,columns={'읍면동'})
#조치원읍/동/면 분리
list_myn = []
list_dong = []
for i in list_d :
  if '조치원읍 ' in i :
    pass;
  elif '동' in i :
    list_dong.append(i)
  else :
    list_myn.append(i)

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
def make_rentfee_df(a,b,c) :
  df = pd.read_csv(a)
  df['보증금(만원)'] = df['보증금(만원)'].str.replace(',','')
  df = df.astype({'보증금(만원)' : float, '계약년월' : 'str'})
  df['시군구'] = df['시군구'].str.replace('세종특별자치시 ','')
  df.replace({'전세' : 'yr', '월세' : 'mth'},inplace=True)
  
  df_j = make_rentfee_data(df,b,'yr')
  df_m = make_rentfee_data(df,b,'mth')
  
  d = pd.merge(c,df_j, how='outer',on='읍면동')
  f = pd.merge(d,df_m, how='outer',on='읍면동')

  return f

def make_rentfee_data(i, b, c) :

  df = i[i['전월세구분'] == c]
  df[['읍면동', '2']] = df['시군구'].str.split(' ', n=1, expand=True)

  df = df.groupby(['읍면동']).agg({'계약일' : 'count', '보증금(만원)':'mean'})
  x = b+'_'+c+'_cnt'
  y = b+'_'+c+'_price'
  
  df.columns = [x,y]

  return df

In [11]:
ofi = make_rentfee_df('data/10.세종시_오피스텔(전월세)_실거래가.csv','ofi',data_d)
apt = make_rentfee_df('data/7.세종시_아파트(전월세)_실거래가.csv','apt',data_d)
ali = make_rentfee_df('data/8.세종시_연립다세대(전월세)_실거래가.csv','ali',data_d)
# 단독주택 제외 >> 대지단위(명확하게 비교하기가 어려움)
summary = pd.merge(ofi, apt, how='outer', on='읍면동')
summary = pd.merge(summary, ali, how='outer', on='읍면동')
market = pd.read_csv('data/14.세종시_상권정보.csv')

C:\Anaconda\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [12]:
def make_summary_by_location(a,b,c) :
    a = summary['읍면동'].str.contains(b)
    a = summary[a]
    a = a.reset_index(drop=True)
    #'전동면'만 제외
    if b == '동' :
        a = a.loc[a['읍면동'] != "전동면"]
        a = a.loc[a['읍면동'] != "연동면"]
        a = a.reset_index(drop=True)
    else :
        a = summary[summary['읍면동'] == '전동면']
        a = summary[summary['읍면동'] == '연동면']

    ex = pd.DataFrame()

    #market divieded
    for i in a['읍면동'] :
        df = market[market['행정동명']==i]
        df = df.groupby('상권업종중분류코드').agg({'행정동명':'count','상권업종중분류명' : 'first','상권업종대분류코드' :'first','상권업종대분류명' : 'first'})
        total = df['행정동명'].sum()
        
        df= df.transpose()
        df['읍면동'] = i
        df['total'] = total

        list_x = df.iloc[0]

        ex = ex.append(pd.Series(list_x), ignore_index=True)
        
    x = pd.merge(a,ex, how='outer', on='읍면동')
    x.head()
    
    return x

In [10]:
code = pd.read_csv('data/15.세종시_상권정보_업종코드.csv')
list_main = list(set(code['대분류코드']))
list_middle = list(set(code['중분류코드']))
list_middle = sorted(list_middle)

In [53]:
dong = make_summary_by_location('dong', '동','1')
dong_final = dong[dong['total'] != 0]

dong_final = dong_final.sort_index(axis=1)
dong_final = dong_final.drop({'ofi_mth_price','ofi_mth_cnt','ofi_yr_price','ofi_yr_cnt'},axis = 1)
print(dong_final.describe())
dong[dong['total'] == 0]['읍면동']

             D01       D02         D03       D04        D05       D06  \
count   9.000000  7.000000    9.000000  4.000000   8.000000  6.000000   
mean    6.111111  2.714286   50.666667  2.000000  12.125000  2.166667   
std     4.512329  1.704336   58.088295  1.414214  10.232127  1.834848   
min     1.000000  1.000000   11.000000  1.000000   3.000000  1.000000   
25%     2.000000  1.000000   16.000000  1.000000   5.000000  1.000000   
50%     6.000000  3.000000   28.000000  1.500000   8.000000  1.000000   
75%     7.000000  4.000000   41.000000  2.500000  15.500000  3.250000   
max    16.000000  5.000000  185.000000  4.000000  32.000000  5.000000   

             D07       D08       D09       D10  ...       R13  ali_mth_cnt  \
count   7.000000  8.000000  6.000000  7.000000  ...  5.000000     3.000000   
mean    5.142857  4.125000  1.166667  3.142857  ...  1.200000    41.000000   
std     4.298394  2.587746  0.408248  1.345185  ...  0.447214    34.655447   
min     1.000000  1.000000  1.

3     다정동
5     합강동
6     가람동
7     반곡동
8     나성동
10    산울동
11    집현동
12    어진동
15    해밀동
Name: 읍면동, dtype: object

In [66]:
check = code[code['대분류코드']=='N']
check['code'] = check['중분류코드'] + check['중분류명']
list_check = list(set(check['code']))
list_check = sorted(list_check)
list_check

<ipython-input-66-4b4dd3542c68>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check['code'] = check['중분류코드'] + check['중분류명']


['N01PC/오락/당구/볼링등',
 'N02무도/유흥/가무',
 'N03연극/영화/극장',
 'N04스포츠/운동',
 'N05요가/단전/마사지',
 'N06전시/관람',
 'N07놀이/여가/취미',
 'N08경마/경륜/성인오락']

In [63]:
for i in list_main:
    df01 = dong_final.loc[:,'ali_mth_cnt':]

    list_y = []
    list_x = [middle for middle in list_middle if i in middle]
    x = list_x[-1]
    print(list_x[0],list_x[-1])
    dong_final.head()
    df02 = dong_final.loc[ :,list_x[0]:list_x[-1]]
    df02['읍면동'] = dong_final['읍면동']
    
    corr = pd.merge(df01,df02,on='읍면동')
    
    print(i, list(set(code[code['대분류코드']==i]['대분류명'])))
    #print(dong[dong > 0.6])

    corr = corr.corr(method = 'pearson')
    #corr.to_csv('png/{}'.format(i)+'.csv')
    df_heatmap = sns.heatmap(corr)
    plt.savefig('png/{}'.format(i)+'.png')

    plt.clf()


O01 O05
O ['숙박']
D01 D26
D ['소매']
R01 R20
R ['학문/교육']
Q01 Q15
Q ['음식']
L01 L05
L ['부동산']
P01 P03
P ['스포츠']
N01 N08
N ['관광/여가/오락']
F01 F20
F ['생활서비스']


<Figure size 1080x576 with 0 Axes>